In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pulp

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('max_colwidth',-1)

# Assignment – Build your fantasy teams for IND v WI 5 match T20 series & find out your avg points per team.

In [2]:
# BATTING POINTS
# run = 1
# boundary B = 1
# six B = 2
# 30 run B = 4
# Half-century B = 8
# Century B = 16
# Duck = -2

# SR:
# >170 = 6
# 150-170 = 4
# 130-150 = 2
# 60-70 = -2
# 50-59.00 = -4
# <50 = -6

# BOWLING POINTS
# wicket = 25
# B(LBW/Bowled) = 8
# 3 w = 4
# 4 w = 8
# 5 w = 16
# maiden = 12

# eco:
# <5 = 6
# 5-5.99 = 4 
# 6-7 = 2
# 10-11 = -2
# 11.01-12 = -4
# >12 = -6

# FIELDING POINTS
# catch = 8
# 3 catch B = 4
# stumping = 12
# RO(direct hit) = 12 
# RO = 6

# OTHER POINTS
# captain = 2x
# vice-captain = 1.5x
# in announced lineups = 4
# playing substitute = 4

### USING FANTACY AVG DATA POINTS FOR PLAYERS FROM THE DATASETS THAT CONSIST OF DREAM11 POINT SYSTEM WHICH IS FIRST FILTERED BY AVARAGE FIRST INNING VENUE SCORE.

# 5th match

In [3]:
df = pd.read_csv('/Users/Shree/Desktop/onceAgain/csv_files/ind_vs_wi/ind_vs_wi_dream11_160to180_5th.csv')

df = df.drop(['bat','bowl'],axis=1)

df = df.join(pd.get_dummies(df['team']))

df = df.join(pd.get_dummies(df['role']))

df['PlayingXI'] = 1

teams = list(df.team.unique())
team1, team2 = teams[0], teams[1]

# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'POINTS'] for obj in df.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) <= 8

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) <= 8

mo.solve()

# output as a column in df
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

fdf = df[df['pickup_status'] == 1].sort_values(['bat', 'wk', 'all', 'bowl'], ascending = False).reset_index(drop = True)

In [4]:
df = df.drop(['bat','bowl'],axis=1)

In [5]:
sum(fdf.credits)

90.0

In [6]:
fdf[['player', 'team', 'role', 'sel', 'credits', 'PlayingXI', 'pickup_status']]

,player,team,role,sel,credits,PlayingXI,pickup_status
0,SA Yadav,ind,bat,84.76,9.0,1,1
1,SO Hetmyer,wi,bat,20.97,8.0,1,1
2,N Pooran,wi,wk,92.68,9.0,1,1
3,HH Pandya,ind,all,92.64,9.0,1,1
4,JO Holder,wi,all,25.14,8.5,1,1
5,KR Mayers,wi,all,53.77,8.0,1,1
6,RL Chase,wi,all,4.15,6.0,1,1
7,YS Chahal,ind,bowl,64.51,8.5,1,1
8,Kuldeep Yadav,ind,bowl,79.34,8.5,1,1
9,AS Joseph,wi,bowl,65.31,8.0,1,1


### Post Match Analysis

In [7]:
fdf[['player', 'PlayingXI','DT']]

,player,PlayingXI,DT
0,SA Yadav,1,1
1,SO Hetmyer,1,0
2,N Pooran,1,1
3,HH Pandya,1,1
4,JO Holder,1,1
5,KR Mayers,1,0
6,RL Chase,1,1
7,YS Chahal,1,0
8,Kuldeep Yadav,1,0
9,AS Joseph,1,0


In [8]:
dream_points5 = 755

In [9]:
points5 = [85,4,74,26,70,17,39,-2,10,-2,136] 
points5 = sum(points5)

# 4th match

In [10]:
df = pd.read_csv('/Users/Shree/Desktop/onceAgain/csv_files/ind_vs_wi/ind_vs_wi_dream11_160to180_4th.csv')

df = df.drop(['bat','bowl'],axis=1)

df = df.join(pd.get_dummies(df['team']))

df = df.join(pd.get_dummies(df['role']))

df['PlayingXI'] = 1

teams = list(df.team.unique())
team1, team2 = teams[0], teams[1]

# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'POINTS'] for obj in df.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) <= 8

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) <= 8

mo.solve()

# output as a column in df
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

fdf = df[df['pickup_status'] == 1].sort_values(['bat', 'wk', 'all', 'bowl'], ascending = False).reset_index(drop = True)

In [11]:
df = df.drop(['bat','bowl'],axis=1)

In [12]:
sum(fdf.credits)

90.5

In [13]:
fdf[['player', 'team', 'role', 'sel', 'credits', 'PlayingXI', 'pickup_status']]

,player,team,role,sel,credits,PlayingXI,pickup_status
0,SA Yadav,ind,bat,84.76,9.0,1,1
1,SO Hetmyer,wi,bat,20.97,8.0,1,1
2,Tilak Varma,ind,bat,81.63,7.5,1,1
3,N Pooran,wi,wk,92.68,9.0,1,1
4,HH Pandya,ind,all,92.64,9.0,1,1
5,JO Holder,wi,all,25.14,8.5,1,1
6,KR Mayers,wi,all,53.77,8.0,1,1
7,YS Chahal,ind,bowl,64.51,8.5,1,1
8,Kuldeep Yadav,ind,bowl,79.34,8.5,1,1
9,R Shepherd,wi,bowl,46.39,7.5,1,1


### Post Match Analysis

In [14]:
fdf[['player', 'PlayingXI','DT']]

,player,PlayingXI,DT
0,SA Yadav,1,0
1,SO Hetmyer,1,1
2,Tilak Varma,1,0
3,N Pooran,1,0
4,HH Pandya,1,0
5,JO Holder,1,0
6,KR Mayers,1,0
7,YS Chahal,1,1
8,Kuldeep Yadav,1,1
9,R Shepherd,1,1


In [15]:
dream_points4 = 703

In [16]:
points4 = [12,88,20,5,4,7,25,29,64,36,10] 
points4 = sum(points4)

# 3rd match

In [17]:
df = pd.read_csv('/Users/Shree/Desktop/onceAgain/csv_files/ind_vs_wi/ind_vs_wi_dream11_160_3rd.csv')

df = df.drop(['bat','bowl'],axis=1)

df = df.join(pd.get_dummies(df['team']))

df = df.join(pd.get_dummies(df['role']))

df['PlayingXI'] = 1

teams = list(df.team.unique())
team1, team2 = teams[0], teams[1]

# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'POINTS'] for obj in df.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) <= 8

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) <= 8

mo.solve()

# output as a column in df
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

fdf = df[df['pickup_status'] == 1].sort_values(['bat', 'wk', 'all', 'bowl'], ascending = False).reset_index(drop = True)

In [18]:
df = df.drop(['bat','bowl'],axis=1)

In [19]:
sum(fdf.credits)

83.5

In [20]:
fdf[['player', 'team', 'role', 'sel', 'credits', 'PlayingXI', 'pickup_status']]

,player,team,role,sel,credits,PlayingXI,pickup_status
0,R Powell,wi,bat,68.37,8.0,1,1
1,YBK Jaiswal,ind,bat,37.60,6.5,1,1
2,SV Samson,ind,wk,12.57,7.5,1,1
3,HH Pandya,ind,all,92.64,9.0,1,1
4,AR Patel,ind,all,27.53,8.0,1,1
5,RL Chase,wi,all,4.15,6.0,1,1
6,Kuldeep Yadav,ind,bowl,79.34,8.5,1,1
7,AS Joseph,wi,bowl,65.31,8.0,1,1
8,Arshdeep Singh,ind,bowl,51.24,7.5,1,1
9,R Shepherd,wi,bowl,46.39,7.5,1,1


### Post Match Analysis

In [21]:
fdf[['player', 'PlayingXI','DT']]

,player,PlayingXI,DT
0,R Powell,1,1
1,YBK Jaiswal,1,0
2,SV Samson,1,0
3,HH Pandya,1,1
4,AR Patel,1,1
5,RL Chase,1,0
6,Kuldeep Yadav,1,1
7,AS Joseph,1,1
8,Arshdeep Singh,1,0
9,R Shepherd,1,0


In [22]:
dream_points3 = 640

In [23]:
points3 = [61,31,31,101,64,5,6,2,10,23,16] 
points3 = sum(points3)

# 2nd match

In [24]:
df = pd.read_csv('/Users/Shree/Desktop/onceAgain/csv_files/ind_vs_wi/ind_vs_wi_dream11_160_2nd.csv')

df = df.drop(['bat','bowl'],axis=1)

df = df.join(pd.get_dummies(df['team']))

df = df.join(pd.get_dummies(df['role']))

df['PlayingXI'] = 1

teams = list(df.team.unique())
team1, team2 = teams[0], teams[1]

# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'POINTS'] for obj in df.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) <= 8

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) <= 8

mo.solve()

# output as a column in df
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

fdf = df[df['pickup_status'] == 1].sort_values(['bat', 'wk', 'all', 'bowl'], ascending = False).reset_index(drop = True)

In [25]:
df = df.drop(['bat','bowl'],axis=1)

In [26]:
sum(fdf.credits)

85.5

In [27]:
fdf[['player', 'team', 'role', 'sel', 'credits', 'PlayingXI', 'pickup_status']]

,player,team,role,sel,credits,PlayingXI,pickup_status
0,R Powell,wi,bat,68.37,8.0,1,1
1,BA King,wi,bat,51.52,7.5,1,1
2,Ishan Kishan,ind,wk,25.68,8.0,1,1
3,HH Pandya,ind,all,92.64,9.0,1,1
4,JO Holder,wi,all,25.14,8.5,1,1
5,AR Patel,ind,all,27.53,8.0,1,1
6,AS Joseph,wi,bowl,65.31,8.0,1,1
7,Arshdeep Singh,ind,bowl,51.24,7.5,1,1
8,R Shepherd,wi,bowl,46.39,7.5,1,1
9,OC McCoy,wi,bowl,13.47,7.0,1,1


### Post Match Analysis

In [28]:
fdf[['player', 'PlayingXI','DT']]

,player,PlayingXI,DT
0,R Powell,1,0
1,BA King,1,0
2,Ishan Kishan,1,1
3,HH Pandya,1,1
4,JO Holder,1,0
5,AR Patel,1,0
6,AS Joseph,1,1
7,Arshdeep Singh,1,1
8,R Shepherd,1,1
9,OC McCoy,1,0


In [29]:
dream_points2 = 755

In [30]:
points2 = [113,55,44,76,62,28,14,14,2,25,2] 
points2 = sum(points2)

# 1st match

In [31]:
df = pd.read_csv('/Users/Shree/Desktop/onceAgain/csv_files/ind_vs_wi/ind_vs_wi_dream11_160_1st.csv')

df = df.drop(['bat','bowl'],axis=1)

df = df.join(pd.get_dummies(df['team']))

df = df.join(pd.get_dummies(df['role']))

df['PlayingXI'] = 1

teams = list(df.team.unique())
team1, team2 = teams[0], teams[1]

# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'POINTS'] for obj in df.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wi']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'ind']] for obj in df.index) <= 8

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'wk']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bat']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'bowl']] for obj in df.index) <= 8

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'all']] for obj in df.index) <= 8

mo.solve()

# output as a column in df
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

fdf = df[df['pickup_status'] == 1].sort_values(['bat', 'wk', 'all', 'bowl'], ascending = False).reset_index(drop = True)

In [32]:
df = df.drop(['bat','bowl'],axis=1)

In [33]:
sum(fdf.credits)

87.5

In [34]:
fdf[['player', 'team', 'role', 'sel', 'credits', 'PlayingXI', 'pickup_status']]

,player,team,role,sel,credits,PlayingXI,pickup_status
0,R Powell,wi,bat,68.37,8.0,1,1
1,BA King,wi,bat,51.52,7.5,1,1
2,Ishan Kishan,ind,wk,25.68,8.0,1,1
3,HH Pandya,ind,all,92.64,9.0,1,1
4,JO Holder,wi,all,25.14,8.5,1,1
5,AR Patel,ind,all,27.53,8.0,1,1
6,Kuldeep Yadav,ind,bowl,79.34,8.5,1,1
7,AS Joseph,wi,bowl,65.31,8.0,1,1
8,Arshdeep Singh,ind,bowl,51.24,7.5,1,1
9,R Shepherd,wi,bowl,46.39,7.5,1,1


### Post Match Analysis

In [35]:
fdf[['player', 'PlayingXI','DT']]

,player,PlayingXI,DT
0,R Powell,1,1
1,BA King,1,1
2,Ishan Kishan,1,0
3,HH Pandya,1,1
4,JO Holder,1,1
5,AR Patel,1,0
6,Kuldeep Yadav,1,0
7,AS Joseph,1,0
8,Arshdeep Singh,1,1
9,R Shepherd,1,1


In [36]:
dream_points1 = 714

In [37]:
points1 = [75,40,53,86,68,66,56,11,36,25,4] 
points1 = sum(points1)

# AVG POINTS PER TEAM

In [42]:
# AVARAGE POINTS PER DREAM TEAM
(dream_points1 + dream_points2 + dream_points3 + dream_points4 + dream_points5) / 5

713.4

In [43]:
# AVARAGE POINT PER TEAM
(points1+points2+points3+points4+points5) / 5

412.4